In [1]:
import requests
import json
import pandas as pd
import apkutils
import os, os.path
import urllib.request

In [2]:
cat_keys = ["TOOLS"] # Can add more than one category
country = "TH"
access_token = "" # Enter your 42matter access token here
date = "07-02-2019" # dd-mm-yyyy

In [3]:
def getJSON(cat_key,country,access_token,date,page):
    URL = "https://data.42matters.com/api/v2.0/android/apps/top_google_charts.json?list_name=topselling_free&cat_key="+ cat_key +"&country="+ country + "&limit=100&access_token=" + access_token + "&page=" + str(page) + "&date=" + date 
    r = requests.get(url = URL)
    data = r.json()
    file_path = "db\\" + date + "\\" + cat_key + "\\json\\"
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    file_path += str(page) + ".json"
    with open(file_path,'w',encoding="utf-8") as fp:
        json.dump(data, fp, indent=4, separators=(',', ': '), sort_keys=True)
    fp.close()
    return data

In [4]:
def getPagePerJSON(DIR):
    return len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

In [5]:
def getAPKList(cat_key,date):
    DIR = "db\\" + date + "\\" + cat_key + "\\json"
    total_page = getPagePerJSON(DIR)
    apk_list = ""
    for p in range(1,total_page+1):
        file_path = "db\\" + date + "\\" + cat_key + "\\json\\" + str(p) + ".json"
        fp = open(file_path,"r",encoding="utf-8")
        data = json.load(fp)
        fp.close()
        for i in range(len(data['app_list'])):
            apk_list += data['app_list'][i]['package_name'] + "\n"
    file_path = "db\\" + date + "\\" + cat_key + "\\apk_list.txt"
    fp = open(file_path,"w",encoding="utf-8")
    fp.write(apk_list)
    fp.close()

In [6]:
def getAPKInfo(cat_key,date):
    DIR = "db\\" + date + "\\" + cat_key + "\\json"
    total_page = getPagePerJSON(DIR)
    for p in range(1,total_page+1):
        file_path = "db\\" + date + "\\" + cat_key + "\\json\\" + str(p) + ".json"
        fp = open(file_path,"r",encoding="utf-8")
        data = json.load(fp)
        fp.close()
        for i in range(len(data['app_list'])):
            package_name = data['app_list'][i]['package_name']
            file_path = "db\\" + date + "\\" + cat_key + "\\apk\\" + package_name + "\\info.json"
            with open(file_path,'w',encoding="utf-8") as fp:
                json.dump(data['app_list'][i], fp, indent=4, separators=(',', ': '), sort_keys=True)
            fp.close()

In [15]:
def getAPK(cat_key,date,isFail):
    if isFail:
        file_path = "db\\" + date + "\\" + cat_key + "\\fail_apk_list.txt"
    else:
        file_path = "db\\" + date + "\\" + cat_key + "\\apk_list.txt"
    fp = open(file_path,"r",encoding="utf-8")
    fail_list = []
    for line in fp:
        package_name = line.split('\n')[0]
        apk_path = "db\\" + date + "\\" + cat_key + "\\apk\\" + package_name + "\\"
        if not os.path.exists(apk_path):
            os.makedirs(apk_path)
        ### Android 5.0
        #err = !gplaycli --device-codename shamu -d {package_name} -f {apk_path}
        ### Android 7.1
        err = !gplaycli --device-codename sailfish -d {package_name} -f {apk_path}
        ### Android 7.0 Tablet
        #err = !gplaycli --device-codename gts3llte -d {package_name} -f {apk_path}
        if(len(err)>0):
            print(package_name,err)
            fail_list.append(line)
    fp.close()
    file_path = "db\\" + date + "\\" + cat_key + "\\fail_apk_list.txt"
    fp = open(file_path,"w",encoding="utf-8")
    data = ""
    for p in fail_list:
        data += p
    fp.write(data)
    fp.close()
    return len(fail_list)

In [21]:
def getScreenshot(cat_key,date):
    screenshot_path = "db\\" + date + "\\" + cat_key + "\\screenshot\\"
    if not os.path.exists(screenshot_path):
        os.makedirs(screenshot_path)
    file_path = "db\\" + date + "\\" + cat_key + "\\apk_list.txt"
    fp = open(file_path,"r",encoding="utf-8")
    app_no = 0
    for line in fp:
        package_name = line.split('\n')[0]
        print(package_name)
        apk_path = "db\\" + date + "\\" + cat_key + "\\apk\\" + package_name + "\\"
        fp2 = open(apk_path + "info.json","r",encoding='utf-8')
        data = json.load(fp2)
        fp2.close()
        pic_no = 1
        for url in data['screenshots']:
            urllib.request.urlretrieve(url, screenshot_path + str(app_no) + "_" + str(pic_no) + ".jpg")
            pic_no += 1
        app_no += 1
    fp.close()

In [9]:
def getAllJSON(cat_keys,date):
    for c in cat_keys:
        p = 1
        data = getJSON(c,country,access_token,date,p)
        while data['has_next']:
            p += 1
            data = getJSON(c,country,access_token,date,p)

In [10]:
def getAllAPK(cat,date,isResume):
    if isResume:
        fail_count = 0
        file_path = "db\\" + date + "\\" + cat + "\\fail_apk_list.txt"
        fp = open(file_path,"r",encoding="utf-8")
        for line in fp:
            fail_count += 1
        fp.close()
    else:
        fail_count = getAPK(cat,date,False)
    if fail_count > 0:
        new_fail_count = getAPK(cat,date,True)
        print(new_fail_count,fail_count)
        while(new_fail_count < fail_count):
            fail_count = new_fail_count
            new_fail_count = getAPK(cat,date,True)
            print(new_fail_count,fail_count)

In [11]:
getAllJSON(cat_keys,date)

In [12]:
getAPKList("TOOLS",date)

In [16]:
#getAllAPK("TOOLS",date,False)
getAllAPK("TOOLS",date,True)

com.samsung.android.voc ["Error while downloading com.samsung.android.voc : 'Your device is not compatible with this item.'", 'A few packages could not be downloaded :', 'com.samsung.android.voc', "'Your device is not compatible with this item.'", '']
com.sec.android.app.voicenote ["Error while downloading com.sec.android.app.voicenote : 'Your device is not compatible with this item.'", 'A few packages could not be downloaded :', 'com.sec.android.app.voicenote', "'Your device is not compatible with this item.'", '']
com.sec.android.app.popupcalculator ["Error while downloading com.sec.android.app.popupcalculator : 'Your device is not compatible with this item.'", 'A few packages could not be downloaded :', 'com.sec.android.app.popupcalculator', "'Your device is not compatible with this item.'", '']
jp.haki.orgnbrowser ['A few packages could not be downloaded :', 'jp.haki.orgnbrowser', "'Item not found.'", '']
com.rsupport.rs.activity.rsupport.aas2 ["Error while downloading com.rsupport

In [17]:
getAPKInfo("TOOLS",date)

In [22]:
getScreenshot("TOOLS",date)

com.camerafit.abdcar.chic
com.noxgroup.app.cleaner
com.google.android.apps.translate
com.lenovo.anyshare.gps
phone.cleaner.speed.booster.cache.clean.android.master
com.junkclean.boost.master
eu.faircode.netguard
com.speedbooster.optimizer
com.oneapp.max.security.pro
com.omni.cleaner
com.truelife.mobile.android.trueiservice
com.ape.easymode1
th.co.crie.tron2.android
com.emoji.coolkeyboard
com.futurebits.zmoji.free
com.apps.go.clean.boost.master
com.jb.emoji.gokeyboard
com.ludashi.dualspace
com.cleanmaster.mguard
com.esrith.pea_mobile
com.dewmobile.kuaiya.play
net.siamdev.nattster.manman
com.groundhog.mcpemaster
eu.tsoml.graphicssettings
ru.zdevs.zarchiver
com.ovelin.guitartuna
com.oneapp.max.cleaner.booster
com.aiofast.cleaner
hotspotshield.android.vpn
com.easytouch.assistivetouch
com.samsung.android.voc
com.mediamushroom.copymydata
com.google.ar.core
com.macbookpro.macintosh.coolsymbols
com.luutinhit.controlcenter
com.google.android.apps.adm
com.gamma.scan
com.onavo.spaceship
org.zwano

com.screenvideorecorder.livestreamfb
com.xentech.deletedphoto.Restore
com.yamee.flashlight.alert.color.flash
com.speed.boost.booster
three.repair.system.fixbugs
com.instantvideosaver.fbvideodownloader
com.blogpentingstudio.bassboosterbluetooth
mobile.datamanager
fastcharge.fastcharging.chargemaster.batterydoctor
com.FengGangApp.Phone.SuperBoost.BoostSuperFastPhone
ultra.game.booster
com.sandwich.booster
com.hilocker
tntstudio.supercompass
com.microphone.bbmic.lite
com.sma.lockscreen.fingerprint
com.dev.assistivetouch
com.litetools.applockpro
coocent.app.tools.soundmeter.noisedetector
infomagicien.cleaner_phone
com.ext.ui
com.realapps.networksignalbooster
com.lionmobi.netmaster
com.screenmirrorapp
com.netease.uu
com.banish.myandroidinfo
com.cloudflare.onedotonedotonedotone
com.eco.bigfont
smart.screen.mirroring.smartviewad.viewstream.tv
proapp.repair.system.fixerror
com.sp999.ram
app.tools.soundmeter.decibel.noisedetector
com.facemoji.lite
com.tomhogenkamp.personalcalc
cpu.gpu.coolerssd

01/02/2019 Fail: (Android 5) 161/540 -> 26/540 -> 21/540 -> (Android 7) 17/540

07/02/2019 Fail: (Android 5) 47/540 -> 17/540 -> 14/540 -> (Android 7) 11/540